In [8]:
import pandas as pd
from pandas import Series,DataFrame
import numpy as np
import os
from sklearn.metrics import mean_squared_error
import datetime
import tensorflow as tf
from tensorflow.contrib import learn
from tensorflow.python.ops import rnn, rnn_cell
import matplotlib.pyplot as plt
import cPickle
import gzip
import seaborn as sns
from sklearn import preprocessing
%matplotlib inline

In [66]:
path = '/home/chocolate/LSTM-source/silver_data/'
data = pd.read_csv(path + 'cleaned_1230-0213.csv')
seq_new_15 = []
data_new = data.ix[:, 2:]
# data_new
#   数据归一化
data_array_per = np.array(data_new)
min_max_scaler = preprocessing.MinMaxScaler()
data_array_per = min_max_scaler.fit_transform(data_array_per)
#  组时间序列
for k in range(len(data_array_per)):
    if k <= len(data_array_per) -5 and k >= 15:
        seq_new_15.append(data_array_per[k - 15:k])
seq_new_15 = np.array(seq_new_15)
print seq_new_15.shape
    

(19742, 15, 5)


In [67]:
data['MeanPrice'] = (data['close'] +data['high']+data['low'])/3.0
ser_1 = data['MeanPrice'][1:]
ser_1.index = range(len(ser_1))
ser1 = data['MeanPrice'][:-1]
ser1.index = range(len(ser1))
data['RaiseDown_1'] = (ser_1 -ser1)/ser1
# 同理，计算当前分钟的后2分钟、3分钟、4分钟、5分钟的涨跌幅(均相对于上一分钟)
ser_2 = data['MeanPrice'][2:]
ser_2.index = range(len(ser_2))
ser2 = ser_1[:-1]
ser2.index = range(len(ser2))
data['RaiseDown_2'] = (ser_2 -ser2)/ser2

ser_3 = data['MeanPrice'][3:]
ser_3.index = range(len(ser_3))
ser3 = ser_2[:-1]
ser3.index = range(len(ser3))
data['RaiseDown_3'] = (ser_3 -ser3)/ser3

ser_4 = data['MeanPrice'][4:]
ser_4.index = range(len(ser_4))
ser4 = ser_3[:-1]
ser4.index = range(len(ser4))
data['RaiseDown_4'] = (ser_4 -ser4)/ser4

ser_5 = data['MeanPrice'][5:]
ser_5.index = range(len(ser_5))
ser5 = ser_4[:-1]
ser5.index = range(len(ser5))
data['RaiseDown_5'] = (ser_5 -ser5)/ser5

In [68]:
# 分5类，按照分类区间分割，分为大涨、小涨、平稳、小跌、大跌
# 表示出涨跌幅位于哪个区间
minute1 = pd.cut(data['RaiseDown_1'],[-100,-0.0003,-0.0001, 0.0001,0.0003,1])
minute2 = pd.cut(data['RaiseDown_2'],[-100,-0.0003,-0.0001, 0.0001,0.0003,1])
minute3 = pd.cut(data['RaiseDown_3'],[-100,-0.0003,-0.0001, 0.0001,0.0003,1])
minute4 = pd.cut(data['RaiseDown_4'],[-100,-0.0003,-0.0001, 0.0001,0.0003,1])
minute5 = pd.cut(data['RaiseDown_5'],[-100,-0.0003,-0.0001, 0.0001,0.0003,1])
pd.concat([minute1,minute2,minute3,minute4,minute5],axis=1)


,RaiseDown_1,RaiseDown_2,RaiseDown_3,RaiseDown_4,RaiseDown_5
0,"(-0.0003, -0.0001]","(0.0003, 1]","(-0.0001, 0.0001]","(0.0001, 0.0003]","(-0.0003, -0.0001]"
1,"(0.0003, 1]","(-0.0001, 0.0001]","(0.0001, 0.0003]","(-0.0003, -0.0001]","(-100, -0.0003]"
2,"(-0.0001, 0.0001]","(0.0001, 0.0003]","(-0.0003, -0.0001]","(-100, -0.0003]","(-0.0003, -0.0001]"
3,"(0.0001, 0.0003]","(-0.0003, -0.0001]","(-100, -0.0003]","(-0.0003, -0.0001]","(-0.0001, 0.0001]"
4,"(-0.0003, -0.0001]","(-100, -0.0003]","(-0.0003, -0.0001]","(-0.0001, 0.0001]","(-0.0001, 0.0001]"
5,"(-100, -0.0003]","(-0.0003, -0.0001]","(-0.0001, 0.0001]","(-0.0001, 0.0001]","(-0.0001, 0.0001]"
6,"(-0.0003, -0.0001]","(-0.0001, 0.0001]","(-0.0001, 0.0001]","(-0.0001, 0.0001]","(-0.0001, 0.0001]"
7,"(-0.0001, 0.0001]","(-0.0001, 0.0001]","(-0.0001, 0.0001]","(-0.0001, 0.0001]","(-0.0001, 0.0001]"
8,"(-0.0001, 0.0001]","(-0.0001, 0.0001]","(-0.0001, 0.0001]","(-0.0001, 0.0001]","(0.0001, 0.0003]"
9,"(-0.0001, 0.0001]","(-0.0001, 0.0001]","(-0.0001, 0.0001]","(0.0001, 0.0003]","(-0.0001, 0.0001]"


In [69]:
# 计算出后5分钟内，涨跌情况分别属于每个区间的数量
data_temp = pd.concat([minute1,minute2,minute3,minute4,minute5],axis=1)
df = DataFrame()
from collections import Counter
for data_temp1 in data_temp.values:
    #print Counter(data333)
    df = pd.concat([df,DataFrame(dict(Counter(data_temp1)),index=[0])],ignore_index=True)
df.head()
# data_per.drop('raise2_count',axis=1,inplace=True)
# df.columns = ['down2_count','down1_count','flat_count','raise1_count','raise2_count']
# data = data_per.join(df)
# data = data.replace(np.nan,0)

,nan,"(-0.0001, 0.0001]","(-0.0003, -0.0001]","(-100, -0.0003]","(0.0001, 0.0003]","(0.0003, 1]"
0,NaN,1,2,NaN,1,1
1,NaN,1,1,1,1,1
2,NaN,1,2,1,1,NaN
3,NaN,1,2,1,1,NaN
4,NaN,2,2,1,NaN,NaN


In [70]:
df1 = df.ix[:,1:]
df1
df1.columns = ['flat_count','down1_count','down2_count','raise1_count','raise2_count']
data = data.join(df1)
data = data.replace(np.nan,0)
data

,Unnamed: 0,datetime,close,high,low,open,volume,MeanPrice,RaiseDown_1,RaiseDown_2,RaiseDown_3,RaiseDown_4,RaiseDown_5,flat_count,down1_count,down2_count,raise1_count,raise2_count
0,0,2016-12-30 09:51:54,39.63,39.63,39.62,39.63,4819394,39.626667,-0.000252,0.001262,-0.000084,0.000168,-0.000252,1,2,0,1,1
1,1,2016-12-30 09:52:56,39.62,39.62,39.61,39.62,4840022,39.616667,0.001262,-0.000084,0.000168,-0.000252,-0.000336,1,1,1,1,1
2,2,2016-12-30 11:15:00,39.67,39.68,39.65,39.67,9336366,39.666667,-0.000084,0.000168,-0.000252,-0.000336,-0.000252,1,2,1,1,0
3,3,2016-12-30 11:16:04,39.66,39.67,39.66,39.66,9438460,39.663333,0.000168,-0.000252,-0.000336,-0.000252,0.000084,1,2,1,1,0
4,4,2016-12-30 11:17:06,39.67,39.68,39.66,39.66,9439228,39.670000,-0.000252,-0.000336,-0.000252,0.000084,-0.000084,2,2,1,0,0
5,5,2016-12-30 11:18:08,39.66,39.67,39.65,39.66,9541758,39.660000,-0.000336,-0.000252,0.000084,-0.000084,0.000084,3,1,1,0,0
6,6,2016-12-30 11:19:12,39.65,39.66,39.63,39.66,9629034,39.646667,-0.000252,0.000084,-0.000084,0.000084,-0.000084,4,1,0,0,0
7,7,2016-12-30 11:20:14,39.63,39.65,39.63,39.64,9694442,39.636667,0.000084,-0.000084,0.000084,-0.000084,-0.000084,5,0,0,0,0
8,8,2016-12-30 11:21:17,39.64,39.65,39.63,39.63,9712584,39.640000,-0.000084,0.000084,-0.000084,-0.000084,0.000168,4,0,0,1,0
9,9,2016-12-30 11:22:21,39.64,39.64,39.63,39.63,9782498,39.636667,0.000084,-0.000084,-0.000084,0.000168,0.000084,4,0,0,1,0


In [72]:
seq_label = []
data_temp = data[14:-5]
data_temp
for k in range(len(data_temp)):
    # 开多仓条件
    if data_temp['raise2_count'].iloc[k] >= 2 and data_temp['down1_count'].iloc[k] <= 2 and data_temp['down2_count'].iloc[k] == 0:
        seq_label.append([1,0,0])
    elif data_temp['raise2_count'].iloc[k] >= 1 and data_temp['down1_count'].iloc[k] <= 1 and data_temp['down2_count'].iloc[k] == 0:
        seq_label.append([1,0,0])
    elif data_temp['raise1_count'].iloc[k] >= 3 and data_temp['down1_count'].iloc[k] <= 1 and data_temp['down2_count'].iloc[k] == 0:
        seq_label.append([1,0,0])
    # 开空仓条件
    elif data_temp['down2_count'].iloc[k] >= 2 and data_temp['raise1_count'].iloc[k] <= 2 and data_temp['raise2_count'].iloc[k] == 0:
        seq_label.append([0,0,1])
    elif data_temp['down2_count'].iloc[k] >= 1 and data_temp['raise1_count'].iloc[k] <= 1 and data_temp['raise2_count'].iloc[k] == 0:
        seq_label.append([0,0,1])
    elif data_temp['down1_count'].iloc[k] >= 3 and data_temp['raise1_count'].iloc[k] <= 1 and data_temp['raise2_count'].iloc[k] == 0:
        seq_label.append([0,0,1])
    else:
        seq_label.append([0,1,0])
        
print u'开多仓：',seq_label.count([1,0,0]),seq_label.count([1,0,0]) /19742.0
print u'不开仓：',seq_label.count([0,1,0]),seq_label.count([0,1,0]) /19742.0
print u'开空仓：',seq_label.count([0,0,1]),seq_label.count([0,0,1]) /19742.0

    

开多仓： 3027 0.153327930301
不开仓： 13638 0.690811467936
开空仓： 3077 0.155860601763


In [82]:
print seq_label

[[0 1 0]
 [0 1 0]
 [0 1 0]
 ..., 
 [0 0 1]
 [0 1 0]
 [0 1 0]]


In [73]:
seq_label = np.array(seq_label)
seq_label.shape

(19742, 3)

In [74]:
from random import shuffle
zip_seq = zip(seq_new_15,seq_label)
shuffle(zip_seq)
seq_new_15,seq_label = zip(*zip_seq)
seq_new_15 = np.array(seq_new_15) 
seq_label = np.array(seq_label)
print seq_new_15.shape, seq_label.shape

(19742, 15, 5) (19742, 3)


In [75]:
train_data = seq_new_15[:14000]
train_label = seq_label[:14000]

test_data = seq_new_15[14000:]
test_label = seq_label[14000:]
train_data.shape, test_label.shape

((14000, 15, 5), (5742, 3))

In [76]:
path_new = '/home/chocolate/LSTM-source/train_test_data/'
f1 = file(path_new + 'train_data_ag_1.npy','wb')
np.save(f1,train_data)
f1.close()
f2 = file(path_new + 'test_data_ag_1.npy','wb')
np.save(f2,test_data)
f2.close()
f3 = file(path_new + 'train_label_ag_1.npy','wb')
np.save(f3,train_label)
f3.close()
f4 = file(path_new + 'test_label_ag_1.npy','wb')
np.save(f4,test_label)
f4.close()

In [77]:
import pandas as pd
from pandas import Series,DataFrame
import numpy as np
import os
import datetime
import tensorflow as tf
from tensorflow.contrib import learn
from tensorflow.python.ops import rnn, rnn_cell
import matplotlib.pyplot as plt
import cPickle
import gzip
from sklearn import preprocessing
%matplotlib inline

In [78]:
path_new = '/home/chocolate/LSTM-source/train_test_data/'
# 读取训练集和测试集
f1 = file(path_new + 'train_data_ag_1.npy','rb')
train_data = np.load(f1)
f2 = file(path_new + 'test_data_ag_1.npy','rb')
test_data = np.load(f2)
f3 = file(path_new + 'train_label_ag_1.npy','rb')
train_label = np.load(f3)
f4 = file(path_new + 'test_label_ag_1.npy','rb')
test_label = np.load(f4)

In [79]:
import random

learning_rate = 0.001 # 学习速率
training_iters = 300000  # 训练总步长
batch_size = 5000 # 每小段步长
display_step = 5  # 每隔100小段显示输出

# 参数设置
n_input = 5 # 特征数量
n_steps = 15 # 时间序列长度
n_hidden = 500 # 隐藏层神经元个数
n_classes = 3 # 分类数量

tf.reset_default_graph()  # 重置流图
# 设置输入输出格式大小
xtr = tf.placeholder("float", [None, n_steps, n_input])
ytr = tf.placeholder("float", [None,n_classes])

# 定义权重和偏移量
weights = {
    'out': tf.Variable(tf.random_normal([n_hidden, n_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([n_classes]))
}

# def pred():
    
def RNN(x, weights, biases):

    # 更改数据集的格式，使其满足RNN网络的输入需求
    # 现在输入格式: (batch_size, n_steps, n_input)
    # 需要的格式: 'n_steps' 个tensor，每个的格式为 (batch_size, n_input)

    # 变换batch_size 和 n_steps的位置
    x = tf.transpose(x, [1, 0, 2])
    # 重新调整格式 (n_steps*batch_size, n_input)
    x = tf.reshape(x, [-1, n_input])
    # 分割成一系列的 'n_steps' 个tensors，每个的格式为 (batch_size, n_input)
    x = tf.split(0, n_steps, x)

    # 定义LSTM神经网络结构
    lstm_cell = rnn_cell.BasicLSTMCell(n_hidden, forget_bias=1.0)
    # 获取输出和状态
    outputs, states = rnn.rnn(lstm_cell, x, dtype=tf.float32)
    # 返回输出
    return tf.matmul(outputs[-1], weights['out']) + biases['out']
# 获取预测值
pred = RNN(xtr, weights, biases)
# train_label = tf.split(0, batch_size, train_label)
# 取下一小段的输入
def next_batch_tr(batch_size):
    step = random.randint(1, len(train_data)/batch_size)
    next_batch_data = train_data[(step-1)*batch_size:step*batch_size]
    next_batch_label = train_label[(step-1)*batch_size:step*batch_size]   
    return next_batch_data, next_batch_label



In [81]:
# 定义代价函数和最优化方法
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, ytr))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# 评估模型准确率
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(ytr, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# 变量初始化
init = tf.initialize_all_variables()
part_saver = tf.train.Saver()
# 创建会话
sess = tf.InteractiveSession()
sess.run(init)
step = 1
    # 一直迭代，直到最大步长
while step * batch_size < training_iters:
    batch_x, batch_y = next_batch_tr(batch_size)
    batch_x = batch_x.reshape((-1, n_steps, n_input))
        # 利用最优化方法调整权值
    sess.run(optimizer, feed_dict={xtr: batch_x, ytr: batch_y})
    if step % display_step == 0:
            # 计算准确率和损失函数
        acc = sess.run(accuracy, feed_dict={xtr: batch_x, ytr: batch_y})
        loss = sess.run(cost, feed_dict={xtr: batch_x, ytr: batch_y})
        # 每隔一定步长显示训练准确率和损失率
        print("Iter " + str(step * batch_size) + ", Minibatch Loss= " +
                "{:.6f}".format(loss) + ", Training Accuracy= " +
                "{:.5f}".format(acc))
    step += 1
print("Optimization Finished!")

saver_path = part_saver.save(sess, "/home/chocolate/LSTM-source/models/model_ag_1/predict.ckpt")
print "Model saved." 


Iter 25000, Minibatch Loss= 1.790085, Training Accuracy= 0.69080
Iter 50000, Minibatch Loss= 0.928160, Training Accuracy= 0.69040
Iter 75000, Minibatch Loss= 0.989835, Training Accuracy= 0.69080
Iter 100000, Minibatch Loss= 0.909599, Training Accuracy= 0.68900
Iter 125000, Minibatch Loss= 0.864804, Training Accuracy= 0.68900
Iter 150000, Minibatch Loss= 0.858459, Training Accuracy= 0.68900
Iter 175000, Minibatch Loss= 0.836751, Training Accuracy= 0.68900
Iter 200000, Minibatch Loss= 0.842377, Training Accuracy= 0.69080
Iter 225000, Minibatch Loss= 0.833716, Training Accuracy= 0.69080
Iter 250000, Minibatch Loss= 0.836446, Training Accuracy= 0.68900
Iter 275000, Minibatch Loss= 0.831909, Training Accuracy= 0.68900
Optimization Finished!
Model saved.


In [83]:
saver = tf.train.Saver()
saver.restore(sess, "/home/chocolate/LSTM-source/models/model_ag_1/predict.ckpt")
print "Model restored."
test_data1 = test_data.reshape((-1, n_steps, n_input))
# test_label1 = test_label[:test_len]
print("Testing Accuracy:",
        sess.run(accuracy, feed_dict={xtr: test_data1, ytr: test_label}))


Model restored.
('Testing Accuracy:', 0.69000345)


In [63]:
data[14:-5]

,Unnamed: 0,datetime,close,high,low,open,volume,MeanPrice,RaiseDown_1,RaiseDown_2,RaiseDown_3,RaiseDown_4,RaiseDown_5,flat_count,down1_count,down2_count,raise1_count,raise2_count
14,14,2016-12-30 11:27:42,39.64,39.65,39.64,39.65,9893006,39.643333,1.681661e-04,-8.406894e-05,-3.363040e-04,8.410429e-05,0.000000e+00,3,0,1,1,0
15,15,2016-12-30 11:28:47,39.65,39.66,39.64,39.65,9897630,39.650000,-8.406894e-05,-3.363040e-04,8.410429e-05,0.000000e+00,0.000000e+00,4,0,1,0,0
16,16,2016-12-30 11:29:51,39.64,39.66,39.64,39.65,9951442,39.646667,-3.363040e-04,8.410429e-05,0.000000e+00,0.000000e+00,-1.681944e-04,3,1,1,0,0
17,17,2016-12-30 11:30:54,39.63,39.64,39.63,39.64,9974756,39.633333,8.410429e-05,0.000000e+00,0.000000e+00,-1.681944e-04,0.000000e+00,4,1,0,0,0
18,18,2016-12-30 11:31:59,39.64,39.64,39.63,39.63,10035034,39.636667,0.000000e+00,0.000000e+00,-1.681944e-04,0.000000e+00,0.000000e+00,4,1,0,0,0
19,19,2016-12-30 11:33:01,39.64,39.64,39.63,39.63,10059762,39.636667,0.000000e+00,-1.681944e-04,0.000000e+00,0.000000e+00,0.000000e+00,4,1,0,0,0
20,20,2016-12-30 11:34:03,39.63,39.65,39.63,39.64,10091450,39.636667,-1.681944e-04,0.000000e+00,0.000000e+00,0.000000e+00,-8.411136e-05,4,1,0,0,0
21,21,2016-12-30 11:35:08,39.63,39.64,39.62,39.63,10153298,39.630000,0.000000e+00,0.000000e+00,0.000000e+00,-8.411136e-05,1.682369e-04,4,0,0,1,0
22,22,2016-12-30 11:36:10,39.63,39.64,39.62,39.63,10186548,39.630000,0.000000e+00,0.000000e+00,-8.411136e-05,1.682369e-04,1.682086e-04,3,0,0,2,0
23,23,2016-12-30 11:37:16,39.63,39.64,39.62,39.63,10224822,39.630000,0.000000e+00,-8.411136e-05,1.682369e-04,1.682086e-04,0.000000e+00,3,0,0,2,0
